In [1]:
import requests
import csv, re, os
import json
import pandas as pd
import numpy as np
from time import sleep
import random

In [2]:
answers = pd.read_csv('weibo/weibo_copy.csv',dtype=object)
savefile = 'weibo/weibo_comments.csv'
savefile_root = 'weibo/weibo_root_count.csv'

In [25]:
## check index!!!!!!
answers = answers.drop_duplicates('mid').reset_index()

In [46]:
with open('weibo/cookie.txt', 'r') as f:
    Cookie = f.readlines()[0]  # Read all lines in the file
    User_Agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15"
    headers = {
            'Cookie': Cookie,
            'User-Agent': User_Agent
        }

In [ ]:
with open(savefile, 'a+') as fc, open(savefile_root, 'a+') as fr:
    header_fc = ['answer_id','answer_content','comment_id','created_time','comment_content','comment_upvote',
              'child_comments_count','author_info','author_gender','author_location','author_following','author_followers']
    writer_fc = csv.DictWriter(fc, delimiter=',', lineterminator='\n',fieldnames=header_fc)
    if not os.path.getsize(savefile):
        writer_fc.writeheader()

    header_fr = ['answer_id','root_comment']
    writer_fr = csv.DictWriter(fr, delimiter=',', lineterminator='\n',fieldnames=header_fr)
    if not os.path.getsize(savefile_root):
        writer_fr.writeheader()
        index_start = 0
    else:
        index_start = len(pd.read_csv(savefile_root))
    print(index_start)
    
    for index in range(index_start, len(answers)):
        try:
            row = answers.loc[index]
        except:
            continue
        page = 0
        root_comment = 0
        mid = row['mid']
        uid = row['author_uid']
        if type(row['comment']) == str:
            url = f'https://weibo.com/ajax/statuses/buildComments?is_reload=1&id={mid}&is_show_bulletin=2&is_mix=0&count=10&uid={uid}&fetch_level=0'
            while True:
                try:
                    res = json.loads(requests.get(url, headers=headers).text)
                except:
                    break
                if res['ok'] == 0:
                    break

                comments = res['data']
                for d in comments:
                    comment_id = d['id']
                    comment_content = re.sub('<.+>', '', d['text'])
                    try:
                        author_location = d['source'].replace('来自','')
                    except:
                        author_location = 'None'
                    author_info = d['user']['profile_url']
                    author_gender = d['user']['gender']
                    author_following = d['user']['friends_count']
                    author_followers = d['user']['followers_count']
                    child_comments_count = d['total_number']
                    comment_like = d['like_counts']
                    writer_fc.writerow({'answer_id':row['mid'], 'answer_content':row['text'],'comment_id':comment_id,
                                     'created_time':d['created_at'],'comment_content':comment_content,'comment_upvote':comment_like,
                                     'child_comments_count':child_comments_count,'author_info':author_info,'author_gender':author_gender,
                                     'author_location':author_location,'author_following':author_following,'author_followers':author_followers})
                root_comment += len(comments)
                max_id = str(res['max_id'])
                if max_id == '0' or len(comments)==0:
                    break
                url = f'https://weibo.com/ajax/statuses/buildComments?flow=0&is_reload=1&id={mid}&is_show_bulletin=2&is_mix=0&max_id={max_id}&count=20&uid={uid}&fetch_level=0'
                sleep(random.uniform(4, 6))
                print(mid, len(comments), root_comment)
        else:
            root_comment = 0
        sleep(random.uniform(4, 6))
        writer_fr.writerow({'answer_id':row['mid'],'root_comment':root_comment})

15562
4778912691526318 19 19
4778912691526318 9 28
4778897294495678 9 9
4778894638453640 14 14
4778853274749730 15 15
4778851626390251 19 19
4778851626390251 20 39
4778851626390251 20 59
4778851626390251 20 79
4778851626390251 19 98
4778851626390251 12 110
4778851626390251 8 118
4778843736376502 12 12
4777779722453791 14 14
4777764035232761 13 13
4777641111979006 19 19
4777461927122579 18 18
4777461927122579 17 35
4777461927122579 17 52
4777461927122579 18 70
4777461927122579 20 90
4777461927122579 18 108
4777461927122579 15 123
4777461927122579 18 141
4777461927122579 20 161
4777461927122579 10 171
4777461927122579 6 177
4777461927122579 9 186
4777461927122579 9 195
4777461927122579 15 210
4777461927122579 9 219
4777461927122579 10 229
4777410601944745 16 16
4777410601944745 19 35
4777410601944745 18 53
4777410601944745 18 71
4777410601944745 14 85
4777338421641594 17 17
4777109555252586 11 11
4776806798591293 16 16
4776806798591293 13 29
4776335593969225 16 16
4776335593969225 19 35
